In [ ]:
import os
import sys
from pathlib import Path

# 1. Define Project Root
# On Colab/Kaggle, this is usually /content/lip-sync-dl-f25
# On PSC, it will be your current folder
PROJECT_ROOT = Path("lip-sync-dl-f25").resolve()
if not PROJECT_ROOT.exists():
    PROJECT_ROOT = Path(".").resolve() # Fallback if already inside

# 2. Clone if needed
if not (PROJECT_ROOT / ".git").exists():
    print("⬇️ Cloning repository...")
    !git clone https://github.com/MUKAMAFrancois/lip-sync-dl-f25.git
    os.chdir("lip-sync-dl-f25")
else:
    print("✅ Repo exists.")
    if PROJECT_ROOT.name == "lip-sync-dl-f25":
        os.chdir(PROJECT_ROOT)

# 3. Fix Nesting (Crucial Step)
# If we are in .../lip-sync-dl-f25/lip-sync-dl-f25, move up
cwd = Path.cwd()
if cwd.name == "lip-sync-dl-f25" and cwd.parent.name == "lip-sync-dl-f25":
    print("⚠️ Nested directory detected. Fixing...")
    os.chdir("..")

print(f"📂 Final Working Directory: {os.getcwd()}")

# 4. Install Deps
os.environ['KAGGLEHUB_CACHE_DIR'] = os.getcwd()
print("📦 Installing Dependencies...")
!pip install -q -r requirements.txt
!pip install -q kagglehub face-alignment pytorch-fid

In [ ]:
import kagglehub, shutil, os
from pathlib import Path

# 1. Run Setup Script (Creates directories)
!python setup_wav2lip.py

# 2. Download Data
print("⬇️ Downloading Dataset...")
path = kagglehub.dataset_download("francoismukama/muavic-german-sample")
target = Path("data/german")
os.makedirs(target, exist_ok=True)

# 3. Move Data
src_root = Path(path) / "mtedx/video/de"
if src_root.exists():
    for split in ["train", "val", "test"]:
        src = src_root / split
        # Fix validation naming if needed
        if not src.exists() and split == "val": src = src_root / "valid"

        dst = target / split
        if src.exists():
            if dst.exists(): shutil.rmtree(dst)
            shutil.copytree(src, dst)
            print(f"✅ Copied {split}")

In [ ]:
print("⚙️ Setting up Wav2Lip Models...")
!python setup_wav2lip.py

In [ ]:
# Now this will work because we overwrote the script in Cell 3!
!python preprocess_training_data.py \
  --data_root data/german \
  --output_root data/german/preprocessed \
  --filelist_root data/german/filelists \
  --no-zip

In [ ]:
!python training/train.py

In [ ]:
import torch
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm
import sys

# Setup Path
if Path("Wav2Lip").exists(): sys.path.append("Wav2Lip")

from models import Wav2Lip
import audio

def run_inference_demo():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Check for model
    ckpts = sorted(Path("checkpoints").glob("*.pth"))
    if not ckpts:
        print("❌ No checkpoint found to test inference.")
        return
    latest_ckpt = ckpts[-1]

    # Load
    model = Wav2Lip().to(device)
    ckpt = torch.load(latest_ckpt, map_location=device)
    model.load_state_dict({k.replace('module.', ''): v for k, v in ckpt['state_dict'].items()})
    model.eval()

    print(f"✅ Model loaded: {latest_ckpt.name}")
    print("   (Inference code ready for full run)")

run_inference_demo()